In [1]:
from datasets import load_dataset
import json

/home/whlin/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_files = {"train": "./data/train.json", "test": "./data/valid.json"}
qa_dataset = load_dataset("json", data_files=data_files)

Using custom data configuration default-bf2c0e59b3d5d02d
Found cached dataset json (/home/whlin/.cache/huggingface/datasets/json/default-bf2c0e59b3d5d02d/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 2/2 [00:00<00:00,  7.71it/s]


In [3]:
# context_dataset = load_dataset('json', data_files = "./data/context.json", cache_dir='./cache/context')
with open("./data/context.json") as f:
    context_data = json.load(f)

print(context_data[0])

鼓是一種打擊樂器，也是一種通訊工具，非洲某些部落用以傳達信息，中國古代軍隊用以發號施令。堅固的女的一面或雙面蒙上拉緊的膜。鼓可以用手或鼓杵敲擊出聲。鼓在非洲的傳統音樂以及在現代音樂中是一種比較重要的樂器，有的樂隊完全由以鼓為主的打擊樂器組成。鼓除了作為樂器外，在古代許多文明中還用鼓來傳播信息。不同類型的鼓，如定音鼓等，均被調校至特定的音調中。更常見的是將不同類型的鼓或打擊樂器互相組合，以構成常於流行音樂出現的爵士鼓。鼓的演奏一般是用手敲擊鼓面，或是用一二隻鼓棒或鼓錘敲擊。由於鼓的觸覺特性及其易於使用，在音樂治療中常用到鼓，特別是手鼓。在許多傳統文化中，鼓有其象徵的意義，也常用在宗教儀式中。像在蒲隆地的卡央達鼓是王權的象徵，卡央達鼓也出現在1962至1966年間的蒲隆地國旗中。在流行音樂或爵士樂中，鼓常常是指由一組鼓及銅鈸組成的爵士鼓，演奏者稱為鼓手。鼓幾乎都有一個圓形的開口，鼓面拉緊後可以固定在上面。但鼓身的形狀就有很多的變化，西洋樂器的鼓，鼓身多半都是圓柱體，但定音鼓的鼓身則是碗形，有些鼓的鼓身則是截角圓錐或是二個接合的截角圓錐。中國、日本、韓國的鼓常常是中間略寬、上下略窄的圓柱體。最早的鼓是出現於西元前六千年的兩河文明。


In [5]:
print(context_data[      6028])

赤壁之戰後，孫權與劉備展開反攻。揚州方面，孫權自己率軍攻打曹操領地合肥，派張昭進攻九江的當塗。守將劉馥與蔣濟的規劃使得孫權屢攻不下合肥而退，張昭的攻勢也失敗。荊州方面，周瑜與劉備率軍攻擊曹仁固守的江陵，其中周瑜負責攻擊江陵，劉備軍關羽負責阻擋曹魏援軍，史稱絕北道。聯軍用了約一年的時間攻克江陵。而劉備則順利的攻佔荊南四郡。孫權為向劉備拉近關係，將其妹嫁給劉備。其後，周瑜曾想出兵攻打益州，達成「竟長江所極」的目標，但他到巴丘後不久即病逝。最後，孫權聽從魯肅的建議，將江陵「借」給劉備，確認他分得荊南四郡，督導荊州，從而得以聯合抵禦曹操。


In [4]:
test_dataset = qa_dataset['train'].select(range(10))

print(test_dataset)
print(test_dataset[0])

Dataset({
    features: ['id', 'question', 'paragraphs', 'relevant', 'answer'],
    num_rows: 10
})
{'id': '593f14f960d971e294af884f0194b3a7', 'question': '舍本和誰的數據能推算出連星的恆星的質量？', 'paragraphs': [2018, 6952, 8264, 836], 'relevant': 836, 'answer': {'start': 108, 'text': '斯特魯維'}}


In [7]:
# def preprocess_0(example):
#     data = dict()
#     data['label'] = example['paragraphs'].index(example['relevant'])
#     # data['question_header_name'] = example['question'][0]
#     for i in range(4):
#         data['sentence{}'.format(str(i))] = context_data[example['paragraphs'][i]]

#     return data

def preprocess_0(example):
    data = dict()
    data['label'] = [example['paragraphs'][i].index(answer_id) for i, answer_id in enumerate(example['relevant'])]
    # there are four possible context for an question
    for i in range(4):
        data['sentence{}'.format(str(i))] = [context_data[paragraph_ids[i]] for paragraph_ids in example['paragraphs']]

    return data

In [12]:
test_dataset.column_names

['id', 'question', 'paragraphs', 'relevant', 'answer']

In [9]:
swag_dataset = test_dataset.map(preprocess_0, batched = True, remove_columns=['answer', 'paragraphs', 'relevant', 'id'])

Loading cached processed dataset at /home/whlin/.cache/huggingface/datasets/json/default-bf2c0e59b3d5d02d/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-c1ba5d4f89ae0179.arrow


In [11]:
swag_dataset[1]

{'question': '在關西鎮以什麼方言為主？',
 'label': 1,
 'sentence0': '開發區依照產業重點的不同分為經濟開發區、經濟技術開發區、工業區等類型，此外還有產業園或工業園、科技園相對專業的區域等，多附屬於開發區。開發區為中國大陸地區推行「改革開放」以來設置最多的經濟區。按照所設層次的不同，分為國家級、省級、縣級三種主要類型，很多鄉鎮也設有開發區。為便於管理，開發區劃定的區域多為地級行政區範圍內跨縣級行政區的區域，或一個縣級行政區範圍劃定的區域，這類開發區以國家級或省級開發區為主；縣級行政區範圍劃定的開發區則以縣級開發區為主，在一個鄉鎮內或跨越鄉鎮劃定區域；鄉鎮一級設立的開發區多跨行政村劃定區域，或一個行政村範圍內劃定區域。',
 'sentence1': '新竹縣是中華民國臺灣省的縣，位於臺灣本島西北部，北臨桃園市，南接苗栗縣，東南以雪山山脈與宜蘭縣、臺中市相連，西部面向台灣海峽，西接與新竹市交界。全縣總面積約1,427平方公里，除鳳山溪、頭前溪中下游沖積平原外，其餘大多為丘陵、台地及山地。早期新竹縣郊區多務農，1970年代工業技術研究院創設於新竹市，1980年代新竹科學工業園區設立於新竹市東區及新竹縣寶山鄉，1990年代位於湖口鄉的新竹工業區也逐漸從傳統產業聚落轉型為新興高科技產業聚落，使得新竹縣成為北台灣的高科技產業重鎮，而人口也在近幾年急速增加。本縣方言於絕大部分地區使用海陸客家話，竹北市及新豐鄉沿海地區部分使用泉州腔閩南話較多，關西鎮及峨眉鄉部分使用四縣腔客家話為主。',
 'sentence2': '新竹縣人口約54萬人，居民以海陸腔客家人為主，閩南人、外省人及原住民為少數，近30年來由於新竹科學園區的發展，也吸引許多外來人口移居。新竹縣目前有中國科技大學新竹校區、明新科技大學、大華科技大學、中華科技大學新竹校區5所大專院校。另外有國立竹東高中、國立竹北高中、、國立關西高中、縣立湖口高中、縣立六家高中、內思高工、忠信高中、東泰高中、義民高中、仰德高中等高中職校。新竹縣在語文閱讀方面也極為推動及重視，曾舉辦過中華民國92年全國語文競賽重大賽事。台灣高鐵穿越本縣，新竹車站設置於竹北市。台鐵縱貫線、六家線、內灣線都有在本縣設站。自2007年3月1日起因興建六家線，內灣線的新竹-竹東間列車停駛4年8個月，內灣線因為六家線完工已於

In [ ]:
print()

In [31]:
ending_names = [f"sentence{i}" for i in range(4)]
context_name = "question"
question_header_name = "question_header_name"
label_column_name = "label"

In [34]:
def preprocess_function(examples):
    first_sentences = [[context] * 4 for context in examples[context_name]]
    print(first_sentences)
    # question_headers = examples[question_header_name]
    # second_sentences = [
    #     [f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)
    # ]
    second_sentences = [
        [f"{examples[end][i]}" for end in ending_names] for i in range(len(examples[context_name]))
    ]
    # for sentence in second_sentences[0]:
    #     print(sentence)
    print(second_sentences)
    labels = examples[label_column_name]
    print(labels)

In [30]:
print(tokenized_dataset[0])

{'question': '舍本和誰的數據能推算出連星的恆星的質量？', 'label': 3, 'sentence0': '1930年，印度物理學家蘇布拉馬尼揚·錢德拉塞卡根據廣義相對論計算出質量大於1.4倍太陽質量的非轉動星體會因重力塌縮成為電子簡併態。愛丁頓雖然在理論上支持黑洞存在的可能性，但同時認為錢德拉塞卡的觀點事實上不能成立，他認為「應當有某種自然定律阻止恆星出現這種荒唐的行為」。當時的物理學家如波耳、亨利·羅素等人都贊同錢德拉塞卡的理論，但出於愛丁頓聲望的原因，他們並沒有公開對錢德拉塞卡表示支持。不過從某種意義上說，愛丁頓也是正確的，當恆星質量大於錢德拉塞卡極限後，確實仍然會有中子簡併壓力阻止恆星繼續塌縮。到了1939年，美國物理學家羅伯特·歐本海默等人推算了這種情形的恆星質量上限，這個極限被稱作托爾曼-歐本海默-沃爾科夫極限。當今的天體物理學家普遍認為，除非有如未知的夸克簡併壓力一類因素的存在，質量大於托爾曼-歐本海默-沃爾科夫極限的恆星將最終會塌縮為錢德拉塞卡所預言的黑洞。即使如此，史瓦西解作為當時能夠描述黑洞行為的唯一精確解，由於具有一些讓人看似不大優美的性質以及難以與實驗觀測相聯繫，一直沒有進入主流物理學研究的視野，關於黑洞的理論乃至整個廣義相對論領域的研究由此擱置了二十年之久。', 'sentence1': '心理學是否為自然科學的範圍，目前也尚存爭議，一般較廣為接受的說法是心理學同時包含在自然科學與社會科學的範疇之中。自然科學的根本目的在於尋找隱藏在自然現象背後的規律，但是自然科學的工作尚不包括研究為什麼會存在這些規律。自然科學認為超自然的、隨意的和自相矛盾的現象是不存在的。自然科學的最重要的兩個支柱是觀察和邏輯推理。由對自然的觀察和邏輯推理自然科學可以引導出大自然中的規律。假如觀察的現象與規律的預言不同，那麼要麼是因為觀察中有錯誤，要麼是因為至此為止被認為是正確的規律是錯誤的。一個超自然因素是不存在的。', 'sentence2': '這是一個長久以來就存在的假設，大多數的恆星都是長期處在特定重力場的多星或聯星系統。特別是許多大質量的歐和逼型恆星，有80%被認為是多星系統的一部分。然而，質量越低的恆星，單獨存在的比例顯然越高，只有25%的紅矮星被發現有伴星。因為85%的恆星是紅矮星，所以在銀河系內多數的恆星都是單獨誕生的。恆星在宇宙中的分

In [36]:
preprocess_function(tokenized_dataset.select(range(2)))

[['舍本和誰的數據能推算出連星的恆星的質量？', '舍本和誰的數據能推算出連星的恆星的質量？', '舍本和誰的數據能推算出連星的恆星的質量？', '舍本和誰的數據能推算出連星的恆星的質量？'], ['在關西鎮以什麼方言為主？', '在關西鎮以什麼方言為主？', '在關西鎮以什麼方言為主？', '在關西鎮以什麼方言為主？']]
[['1930年，印度物理學家蘇布拉馬尼揚·錢德拉塞卡根據廣義相對論計算出質量大於1.4倍太陽質量的非轉動星體會因重力塌縮成為電子簡併態。愛丁頓雖然在理論上支持黑洞存在的可能性，但同時認為錢德拉塞卡的觀點事實上不能成立，他認為「應當有某種自然定律阻止恆星出現這種荒唐的行為」。當時的物理學家如波耳、亨利·羅素等人都贊同錢德拉塞卡的理論，但出於愛丁頓聲望的原因，他們並沒有公開對錢德拉塞卡表示支持。不過從某種意義上說，愛丁頓也是正確的，當恆星質量大於錢德拉塞卡極限後，確實仍然會有中子簡併壓力阻止恆星繼續塌縮。到了1939年，美國物理學家羅伯特·歐本海默等人推算了這種情形的恆星質量上限，這個極限被稱作托爾曼-歐本海默-沃爾科夫極限。當今的天體物理學家普遍認為，除非有如未知的夸克簡併壓力一類因素的存在，質量大於托爾曼-歐本海默-沃爾科夫極限的恆星將最終會塌縮為錢德拉塞卡所預言的黑洞。即使如此，史瓦西解作為當時能夠描述黑洞行為的唯一精確解，由於具有一些讓人看似不大優美的性質以及難以與實驗觀測相聯繫，一直沒有進入主流物理學研究的視野，關於黑洞的理論乃至整個廣義相對論領域的研究由此擱置了二十年之久。', '心理學是否為自然科學的範圍，目前也尚存爭議，一般較廣為接受的說法是心理學同時包含在自然科學與社會科學的範疇之中。自然科學的根本目的在於尋找隱藏在自然現象背後的規律，但是自然科學的工作尚不包括研究為什麼會存在這些規律。自然科學認為超自然的、隨意的和自相矛盾的現象是不存在的。自然科學的最重要的兩個支柱是觀察和邏輯推理。由對自然的觀察和邏輯推理自然科學可以引導出大自然中的規律。假如觀察的現象與規律的預言不同，那麼要麼是因為觀察中有錯誤，要麼是因為至此為止被認為是正確的規律是錯誤的。一個超自然因素是不存在的。', '這是一個長久以來就存在的假設，大多數的恆星都是長期處在特定重力場的多星或聯星系統。特別是許多大質量的歐和逼型恆星，有80%被認為是多星系統的一部分。然

In [31]:
print(tokenized_dataset)
print(tokenized_dataset[0]['question'])
for i in range(4):
    print(tokenized_dataset[0]["sentence{}".format(str(i))])
print(tokenized_dataset[0]['label'])

Dataset({
    features: ['question', 'label', 'question_header_name', 'sentence0', 'sentence1', 'sentence2', 'sentence3'],
    num_rows: 10
})
舍本和誰的數據能推算出連星的恆星的質量？
1930年，印度物理學家蘇布拉馬尼揚·錢德拉塞卡根據廣義相對論計算出質量大於1.4倍太陽質量的非轉動星體會因重力塌縮成為電子簡併態。愛丁頓雖然在理論上支持黑洞存在的可能性，但同時認為錢德拉塞卡的觀點事實上不能成立，他認為「應當有某種自然定律阻止恆星出現這種荒唐的行為」。當時的物理學家如波耳、亨利·羅素等人都贊同錢德拉塞卡的理論，但出於愛丁頓聲望的原因，他們並沒有公開對錢德拉塞卡表示支持。不過從某種意義上說，愛丁頓也是正確的，當恆星質量大於錢德拉塞卡極限後，確實仍然會有中子簡併壓力阻止恆星繼續塌縮。到了1939年，美國物理學家羅伯特·歐本海默等人推算了這種情形的恆星質量上限，這個極限被稱作托爾曼-歐本海默-沃爾科夫極限。當今的天體物理學家普遍認為，除非有如未知的夸克簡併壓力一類因素的存在，質量大於托爾曼-歐本海默-沃爾科夫極限的恆星將最終會塌縮為錢德拉塞卡所預言的黑洞。即使如此，史瓦西解作為當時能夠描述黑洞行為的唯一精確解，由於具有一些讓人看似不大優美的性質以及難以與實驗觀測相聯繫，一直沒有進入主流物理學研究的視野，關於黑洞的理論乃至整個廣義相對論領域的研究由此擱置了二十年之久。
心理學是否為自然科學的範圍，目前也尚存爭議，一般較廣為接受的說法是心理學同時包含在自然科學與社會科學的範疇之中。自然科學的根本目的在於尋找隱藏在自然現象背後的規律，但是自然科學的工作尚不包括研究為什麼會存在這些規律。自然科學認為超自然的、隨意的和自相矛盾的現象是不存在的。自然科學的最重要的兩個支柱是觀察和邏輯推理。由對自然的觀察和邏輯推理自然科學可以引導出大自然中的規律。假如觀察的現象與規律的預言不同，那麼要麼是因為觀察中有錯誤，要麼是因為至此為止被認為是正確的規律是錯誤的。一個超自然因素是不存在的。
這是一個長久以來就存在的假設，大多數的恆星都是長期處在特定重力場的多星或聯星系統。特別是許多大質量的歐和逼型恆星，有80%被認為是多星系統的一部分。然而，質量越低的恆星，

In [24]:
raw_datasets = load_dataset("swag")

No config specified, defaulting to: swag/regular
Found cached dataset swag (/home/whlin/.cache/huggingface/datasets/swag/regular/0.0.0/9640de08cdba6a1469ed3834fcab4b8ad8e38caf5d1ba5e7436d8b1fd067ad4c)
100%|██████████| 3/3 [00:00<00:00, 1419.39it/s]


In [37]:
if raw_datasets["train"] is not None:
    column_names = raw_datasets["train"].column_names
else:
    column_names = raw_datasets["validation"].column_names    
ending_names = [f"ending{i}" for i in range(4)]
context_name = "sent1"
question_header_name = "sent2"
label_column_name = "label" if "label" in column_names else "labels"

In [38]:
def preprocess_function(examples):
    first_sentences = [[context] * 4 for context in examples[context_name]]
    print(first_sentences)
    question_headers = examples[question_header_name]
    # print(question_headers)
    second_sentences = [
        [f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)
    ]
    print(second_sentences)
    labels = examples[label_column_name]
    print(labels)

In [40]:
preprocess_function(raw_datasets["train"].select(range(2)))

[['Members of the procession walk down the street holding small horn brass instruments.', 'Members of the procession walk down the street holding small horn brass instruments.', 'Members of the procession walk down the street holding small horn brass instruments.', 'Members of the procession walk down the street holding small horn brass instruments.'], ['A drum line passes by walking down the street playing their instruments.', 'A drum line passes by walking down the street playing their instruments.', 'A drum line passes by walking down the street playing their instruments.', 'A drum line passes by walking down the street playing their instruments.']]
[['A drum line passes by walking down the street playing their instruments.', 'A drum line has heard approaching them.', "A drum line arrives and they're outside dancing and asleep.", 'A drum line turns the lead singer watches the performance.'], ['Members of the procession are playing ping pong and celebrating one left each in quick.', 